In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import ftfy
import pickle
from datetime import datetime

In [5]:
df = pd.read_json("messages_war_monitor.json")

In [6]:
df

,id,date,sender_id,message
0,27572,2025-04-20T21:00:26+00:00,-1001641260594,📡 **Обстановка станом на 00:00 21.04.25**\n\n—...
1,27573,2025-04-20T21:07:44+00:00,-1001641260594,Доба (20-21.04.25) пройшла без повітряних загр...
2,27574,2025-04-20T23:04:57+00:00,-1001641260594,👁 Розвідувальний БпЛА працює в Корюківському р...
3,27575,2025-04-20T23:23:23+00:00,-1001641260594,❗️ Пуски планеруючих авіаційних бомб (КАБ) по ...
4,27576,2025-04-20T23:43:00+00:00,-1001641260594,⚠️ **Загроза ударних БпЛА з Північного Сходу.\...
5,27577,2025-04-21T00:00:06+00:00,-1001641260594,Сумська область:\n5х БпЛА від Білопілля у напр...
6,27578,2025-04-21T00:00:51+00:00,-1001641260594,Чернігівська область:\n6х БпЛА ланкою від Бату...
7,27579,2025-04-21T00:05:29+00:00,-1001641260594,"⚠️ Група БпЛА через Запорізький район, Гуляйпо..."
8,27580,2025-04-21T00:11:18+00:00,-1001641260594,⚠️ Група БпЛА з Луганщини заходить на Харківщи...
9,27581,2025-04-21T00:27:23+00:00,-1001641260594,03:26 Загальна.\n\nЧернігівська область:\n3х Б...


In [621]:
df = df.drop('id', axis=1)

In [623]:
df = df.drop('sender_id', axis=1)

In [625]:
df

,date,message
0,2022-05-01 18:47:53+00:00,UPD. \n- Два БДК находятся у западного берега ...
1,2022-05-02 07:02:30+00:00,"Два катера ""Раптор"" уничтожены у берегов остро..."
2,2022-05-02 08:12:12+00:00,UPD.\nМестоположение подбитых катеров
3,2022-05-03 15:27:03+00:00,❗️ Вылет стратегических бомбардировщиков ТУ-95...
4,2022-05-04 18:19:25+00:00,❗️ Сообщают о вылетах в сторону Николаева.\n\n...
...,...,...
25351,2025-03-01 22:13:54+00:00,00:13\n\nПо залишках.\n\nНа Житомирщині 1х БпЛ...
25352,2025-03-01 22:48:40+00:00,00:48\n\nЖитомирська область:\n3х БпЛА через Ж...
25353,2025-03-01 23:05:14+00:00,Загроза балістики зі Сходу. Увага
25354,2025-03-01 23:34:27+00:00,01:34\nЗалишки БпЛА прямують на Хмельниччину а...


In [627]:
df['timestamp'] = pd.to_datetime(df['date'])
df['date'] = df['timestamp'].dt.date.astype(str)
df['time'] = df['timestamp'].dt.time.astype(str)

In [629]:
df['date'] = pd.to_datetime(df['date'])
df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S').dt.time

In [631]:
df

,date,message,timestamp,time
0,2022-05-01,UPD. \n- Два БДК находятся у западного берега ...,2022-05-01 18:47:53+00:00,18:47:53
1,2022-05-02,"Два катера ""Раптор"" уничтожены у берегов остро...",2022-05-02 07:02:30+00:00,07:02:30
2,2022-05-02,UPD.\nМестоположение подбитых катеров,2022-05-02 08:12:12+00:00,08:12:12
3,2022-05-03,❗️ Вылет стратегических бомбардировщиков ТУ-95...,2022-05-03 15:27:03+00:00,15:27:03
4,2022-05-04,❗️ Сообщают о вылетах в сторону Николаева.\n\n...,2022-05-04 18:19:25+00:00,18:19:25
...,...,...,...,...
25351,2025-03-01,00:13\n\nПо залишках.\n\nНа Житомирщині 1х БпЛ...,2025-03-01 22:13:54+00:00,22:13:54
25352,2025-03-01,00:48\n\nЖитомирська область:\n3х БпЛА через Ж...,2025-03-01 22:48:40+00:00,22:48:40
25353,2025-03-01,Загроза балістики зі Сходу. Увага,2025-03-01 23:05:14+00:00,23:05:14
25354,2025-03-01,01:34\nЗалишки БпЛА прямують на Хмельниччину а...,2025-03-01 23:34:27+00:00,23:34:27


In [633]:
df = df.drop('timestamp', axis=1)

In [635]:
df

,date,message,time
0,2022-05-01,UPD. \n- Два БДК находятся у западного берега ...,18:47:53
1,2022-05-02,"Два катера ""Раптор"" уничтожены у берегов остро...",07:02:30
2,2022-05-02,UPD.\nМестоположение подбитых катеров,08:12:12
3,2022-05-03,❗️ Вылет стратегических бомбардировщиков ТУ-95...,15:27:03
4,2022-05-04,❗️ Сообщают о вылетах в сторону Николаева.\n\n...,18:19:25
...,...,...,...
25351,2025-03-01,00:13\n\nПо залишках.\n\nНа Житомирщині 1х БпЛ...,22:13:54
25352,2025-03-01,00:48\n\nЖитомирська область:\n3х БпЛА через Ж...,22:48:40
25353,2025-03-01,Загроза балістики зі Сходу. Увага,23:05:14
25354,2025-03-01,01:34\nЗалишки БпЛА прямують на Хмельниччину а...,23:34:27


In [641]:
df.dtypes

date       datetime64[ns]
message            object
time               object
dtype: object

In [643]:
df['message']

0        UPD. \n- Два БДК находятся у западного берега ...
1        Два катера "Раптор" уничтожены у берегов остро...
2                    UPD.\nМестоположение подбитых катеров
3        ❗️ Вылет стратегических бомбардировщиков ТУ-95...
4        ❗️ Сообщают о вылетах в сторону Николаева.\n\n...
                               ...                        
25351    00:13\n\nПо залишках.\n\nНа Житомирщині 1х БпЛ...
25352    00:48\n\nЖитомирська область:\n3х БпЛА через Ж...
25353                    Загроза балістики зі Сходу. Увага
25354    01:34\nЗалишки БпЛА прямують на Хмельниччину а...
25355                          ⚪️Відбій загрози балістики.
Name: message, Length: 25356, dtype: object

In [645]:
existing_dates = pd.Series(df['date'].unique())

In [647]:
existing_dates

0      2022-05-01
1      2022-05-02
2      2022-05-03
3      2022-05-04
4      2022-05-05
          ...    
1025   2025-02-25
1026   2025-02-26
1027   2025-02-27
1028   2025-02-28
1029   2025-03-01
Length: 1030, dtype: datetime64[ns]

In [649]:
start_range = datetime(2022, 5, 1)
end_range = datetime(2025, 3, 1)

In [651]:
full_range = pd.date_range(start=start_range, end=end_range, freq='D')

In [653]:
missing_dates = full_range.difference(existing_dates)

In [655]:
missing_dates

DatetimeIndex(['2022-05-19', '2023-01-09', '2023-01-11', '2023-01-12',
               '2023-01-13', '2023-01-15'],
              dtype='datetime64[ns]', freq=None)

In [657]:
"""
DO NOT EXIST:
Russian Offensive Campaign Assessment, November 24, 2022
Russian Offensive Campaign Assessment, December 25, 2022
Russian Offensive Campaign Assessment, January 1, 2023
Russian Offensive Campaign Assessment, November 23, 2023
Russian Offensive Campaign Assessment, December 25, 2023
Russian Offensive Campaign Assessment, January 1, 2024
Russian Offensive Campaign Assessment, October 8, 2024
Russian Offensive Campaign Assessment, November 28, 2024
Russian Offensive Campaign Assessment, December 25, 2024
Russian Offensive Campaign Assessment, January 1, 2025
"""
!!!!!!переписати

['zsh:1: command not found: !!!!переписати']

In [659]:
df.tail(20)

,date,message,time
25336,2025-03-01,❗️ Запоріжжя — БпЛА через місто.,17:42:26
25337,2025-03-01,Ворог ударив по житловій багатоповерхівці у З...,17:59:55
25338,2025-03-01,⚠️ **Загроза ударних БпЛА з Південного та Схід...,18:34:48
25339,2025-03-01,21:40\n\n⚠️ Більшість БпЛА через Полтавщину.\n...,19:40:45
25340,2025-03-01,22:23\n\nБез помітних змін.\nБільшість БпЛА на...,20:23:45
25341,2025-03-01,⚠️ 1х БпЛА на Бровари зі сходу.,20:52:53
25342,2025-03-01,"Київ і агломерація трохи відпочиває, найближчі...",21:05:49
25343,2025-03-01,23:09\n\nБпЛА значно поменшало.\n\n3х БпЛА чер...,21:09:17
25344,2025-03-01,⚠️ 1х БпЛА повз Обухів у напрямку Києва.,21:15:12
25345,2025-03-01,❗️ Київ — 1х БпЛА на Дарницький район.\nПовз Щ...,21:26:47


In [662]:
df = df.groupby('date').agg({
    'message': lambda x: '\n'.join(x),
    'time': 'last' 
}).reset_index()

In [664]:
df

,date,message,time
0,2022-05-01,UPD. \n- Два БДК находятся у западного берега ...,18:47:53
1,2022-05-02,"Два катера ""Раптор"" уничтожены у берегов остро...",08:12:12
2,2022-05-03,❗️ Вылет стратегических бомбардировщиков ТУ-95...,15:27:03
3,2022-05-04,❗️ Сообщают о вылетах в сторону Николаева.\n\n...,19:35:36
4,2022-05-05,❗️3:23 Запуск крылатых ракет х-101 над Каспийс...,01:08:50
...,...,...,...
1025,2025-02-25,⚠️ Станом на 02:30 4х борти Ту-95мс прямують у...,23:09:55
1026,2025-02-26,Останні два БпЛА на Полтавщині у Полтавському ...,23:07:22
1027,2025-02-27,"02:35\n\nБпЛА меншає, більшість збивається в о...",23:55:08
1028,2025-02-28,⚠️ Київ — залишає Подільський район вектором н...,23:24:12


In [666]:
dt = pd.read_json("messages_hromadske_ua.json")

In [668]:
dt

,id,date,sender_id,message
0,16198,2022-02-24 03:28:28+00:00,-1001059792804,⚡️Путін [заявив](https://hromadske.ua/posts/pu...
1,16199,2022-02-24 03:48:57+00:00,-1001059792804,❗️Жителі низки міст України [повідомляють](htt...
2,16200,2022-02-24 04:14:13+00:00,-1001059792804,«Путін розпочав повномасштабну війну проти Укр...
3,16201,2022-02-24 04:47:58+00:00,-1001059792804,⚡️Зеленський [заявив](https://hromadske.ua/pos...
4,16202,2022-02-24 05:02:29+00:00,-1001059792804,"Ми ведемо текстову трансляцію, куди збираємо і..."
...,...,...,...,...
4804,21025,2022-04-30 20:29:14+00:00,-1001059792804,"__«У них ще залишаються ракети, щоб стріляти п..."
4805,21026,2022-04-30 20:50:18+00:00,-1001059792804,[​​](https://telegra.ph/file/6699083c236d0d1bb...
4806,21027,2022-04-30 22:18:21+00:00,-1001059792804,🇪🇺 Євросоюз [високо оцінив](https://hromadske....
4807,21028,2022-04-30 22:59:05+00:00,-1001059792804,🔥 На Херсонщині ЗСУ [знищили](https://hromadsk...


In [670]:
dt = dt.drop('id', axis=1)

In [672]:
dt = dt.drop('sender_id', axis=1)

In [674]:
dt['timestamp'] = pd.to_datetime(dt['date'])
dt['date'] = dt['timestamp'].dt.date.astype(str)
dt['time'] = dt['timestamp'].dt.time.astype(str)

In [676]:
dt['date'] = pd.to_datetime(dt['date'])

In [678]:
dt.dtypes

date              datetime64[ns]
message                   object
timestamp    datetime64[ns, UTC]
time                      object
dtype: object

In [680]:
dt = dt.drop('timestamp', axis=1)

In [688]:
dt = dt.groupby('date').agg({
    'message': lambda x: '\n'.join(x),
    'time': 'last' 
}).reset_index()

In [690]:
dt.tail(20)

,date,message,time
46,2022-04-11,⛪️Майже 200 священників УПЦ Московського патрі...,23:52:48
47,2022-04-12,🇬🇧🇺🇸 У США та Великій Британії [відреагували](...,23:02:14
48,2022-04-13,👊🏻 Зенітні ракетні війська Повітряних сил ЗСУ ...,23:25:47
49,2022-04-14,🤞 Україна наразі [веде перемовини](https://hro...,23:41:47
50,2022-04-15,На Херсонщині з полону повернули 5 українських...,23:40:54
51,2022-04-16,💑З початку повномасштабної війни в Україні одр...,23:54:48
52,2022-04-17,⛽️ Наступний пакет санкцій Євросоюзу проти рос...,23:49:47
53,2022-04-18,У Боярці Київської області може [з'явитися](ht...,23:53:51
54,2022-04-19,У Чорному морі російські ракетні та десантні к...,23:58:35
55,2022-04-20,окупанти примусово переселили в росію 500 тися...,23:55:34


In [692]:
combined_df = pd.concat([dt, df], ignore_index=True)

In [694]:
combined_df


,date,message,time
0,2022-02-24,⚡️Путін [заявив](https://hromadske.ua/posts/pu...,23:01:19
1,2022-02-25,[​​](https://telegra.ph/file/f7bd512a24f6e63cb...,23:56:20
2,2022-02-26,⚡️У Києві за вчора [вбили](https://www.faceboo...,23:53:24
3,2022-02-27,⚡️У Харкові від обстрілів [постраждав](https:/...,23:57:28
4,2022-02-28,«Переговори підуть або в потрібному 🇺🇦 напрямк...,23:09:19
...,...,...,...
1091,2025-02-25,⚠️ Станом на 02:30 4х борти Ту-95мс прямують у...,23:09:55
1092,2025-02-26,Останні два БпЛА на Полтавщині у Полтавському ...,23:07:22
1093,2025-02-27,"02:35\n\nБпЛА меншає, більшість збивається в о...",23:55:08
1094,2025-02-28,⚠️ Київ — залишає Подільський район вектором н...,23:24:12


In [1]:
df = pd.read_csv('telegram_data_finalversion.csv')

NameError: name 'pd' is not defined

In [53]:
lemmatizer = WordNetLemmatizer()

In [54]:
# nltk.download()

In [55]:

stop_words = set(stopwords.words('english'))
stop_words = set(stopwords.words('russian'))

# --- Custom Domain-Specific Stopwords ---
custom_stops = {
    # --- Metadata / Reporting / Channel Specific ---
    "повідомлення", "сообщение", "повідомляють", "сообщают", "інформація", "информация", "офіційно", "официально",
    "неофіційно", "неофициально", "підтверджено", "подтверждено", "непідтверджено", "неподтверждено", "оновлення", "обновление",
    "доповнення", "дополнение", "дані", "данные", "станом", "состоянию", "джерело", "источник", "згідно", "согласно",
    "як", "как", "без",
    "канал", "channel", "телеграм", "телеграмм", "telegram", "пост", "post", "публікація", "публикация",
    "репост", "пересилка", "forwarded", "fwd", "увага", "внимание", "важливо", "важно", "терміново", "срочно",
    "екстрено", "экстренно", "звіт", "отчет", "огляд", "обзор", "ситуація", "ситуация", "карта", "карты",
    "актуальна", "актуальная", "інтерактивна", "интерактивная", "карта", "карті", "карте",
    "попередньо", "предварительно", "уточнюється", "уточняется", "деталі", "детали", "подробиці", "подробности",
    "відомо", "известно", "пишуть", "пишут", "кажуть", "говорят", "заявили", "заявил", "заявила", "заявило",
    "коментар", "комментарий", "цитата", "заява", "заявление", "інтерв'ю", "интервью", "брифінг", "брифинг",
    "підсумок", "итог", "підсумки", "итоги", "аналіз", "анализ", "оцінка", "оценка", "факт", "від", "от",
    "фактчекінг", "фактчекинг", "спростування", "опровержение", "реакція", "реакция", "читати", "читать", "дивитись", "смотреть",
    "далі", "далее", "тут", "здесь", "нижче", "ниже", "вище", "выше", "посилання", "ссылка", "лінк", "линк",
    "продовження", "продолжение", "початок", "начало", "кінець", "конец", "частина", "часть", "номер", "№", "no",
    "фото", "відео", "аудіо", "photo", "video", "audio", "скріншот", "скриншот", "screenshot", "UPD", "upd", "станом", "обстановка",

    # --- Generic Time References ---
    "година", "годин", "год", "час", "часов", "хвилина", "хвилин", "хв", "минута", "минут", "мин",
    "секунда", "секунд", "сек",
    "день", "дня", "днів", "день", "дня", "дней",
    "ніч", "ночі", "ночей", "ночь", "ночи", "ночей",
    "ранок", "ранку", "утро", "утра", "вечір", "вечора", "вечер", "вечера",
    "сьогодні", "сегодня", "вчора", "вчера", "завтра",
    "нещодавно", "недавно", "зранку", "утром", "вдень", "днем",
    "ввечері", "вечером", "вночі", "ночью", "дата",
    "місяць", "месяц", "рік", "год", "тиждень", "неделя",
    "минулий", "прошлый", "наступний", "следующий", "поточний", "текущий", "зараз", "сейчас", "тепер", "теперь",
    "потім", "потом", "доба", "сутки", "годинник", "часы", "календар", "календарь",
    "понеділок", "вівторок", "середа", "четвер", "п'ятниця", "субота", "неділя",
    "понедельник", "вторник", "среда", "четверг", "пятница", "суббота", "воскресенье",
    "січень", "лютий", "березень", "квітень", "травень", "червень", "липень", "серпень", "вересень", "жовтень", "листопад", "грудень",
    "январь", "февраль", "март", "апрель", "май", "июнь", "июль", "август", "сентябрь", "октябрь", "ноябрь", "декабрь",
    "близько", "около", "приблизно", "приблизительно", "орієнтовно", "ориентировочно", "біля", "около",
    "РФ","назад", "тому",

    # --- Generic Verbs (Base forms, consider lemmatization for variants) ---
    "бути", "быть", "мати", "иметь", "робити", "делать", "зробити", "сделать", "могти", "мочь", "вміти", "уметь",
    "сказати", "сказать", "говорити", "говорить", "повідомляти", "сообщать", "повідомити", "сообщить",
    "тривати", "продолжаться", "продовжувати", "продолжать", "починати", "начинать", "почати", "начать",
    "закінчувати", "заканчивать", "закінчити", "закончить", "знаходитись", "находиться", "перебувати", "пребывать",
    "відбуватись", "происходить", "стати", "стать","чути", "слышать", "давати", "давать", "дати", "дать", "брати", "брать", "взяти", "взять", "отримувати", "получать",
    "отримати", "получить", "використовувати", "использовать", "застосовувати", "применять", "здійснювати", "осуществлять",
    "працювати", "работать", "діяти", "действовать", "залишатись", "оставаться", "залишити", "оставить",
    "очікувати", "ожидать", "очікується", "ожидается", "рухатись", "двигаться", "йти", "идти", "їхати", "ехать",
    "летіти", "лететь", "прибувати", "прибывать", "виглядати", "выглядеть", "намагатись", "пытаться", "пробувати", "пробовать",
    "вважати", "считать", "називати", "называть","заявляти", "заявлять", "фіксувати", "фиксировать", "спостерігати", "наблюдать", "означати", "означать", "значити", "значить",
    "підтверджувати", "подтверждать", "спростовувати", "опровергать", "зберігати", "сохранять", "захищати", "защищать",
    "атакувати", "атаковать", "вести","включати", "включать", "містити", "содержать",


    # --- Generic Nouns (Be cautious, some might be features) ---
    "район", "область", "місто", "село", "населений", "пункт",
    "територія", "территория", "регіон", "регион", "країна", "страна", "місце", "место", "зона", "зона",
    "напрямок", "направление", "бік", "сторона", "частина", "часть", "ділянка", "участок", "сектор", "сектор",
    "тип", "тип", "вид", "вид", "різновид", "разновидность", "рівень", "уровень", "ступінь", "степень",
    "кількість", "количество", "номер", "номер", "число", "число", "група", "группа", "особа", "лицо", "люди", "люди", 
    "населення", "население", "мешканці", "жители", "громада",
    "засіб", "средство", "об'єкт", "объект", "предмет", "предмет", "питання", "вопрос", "тема", "тема", 
    "причина", "причина", "наслідок", "последствие", "результат", "результат", "вихід", "выход", "вхід", "вход",
    "шлях", "путь", "дорога", "дорога", "мета", "цель", "завдання", "задача", "план", "план", 
    "дія", "действие", "подія", "событие", "випадок", "случай", "можливість", "возможность", "здатність", "способность",
    "стан", "состояние", "статус", "статус", "зміна", "изменение", "процес", "процесс", "розвиток", "развитие",
    "підтримка", "поддержка", "допомога", "помощь", "ресурс", "ресурс", "потреба", "необходимость", "потребность",
    "влада", "власть", "уряд", "правительство", "організація", "организация", "служба",
    "захід", "мероприятие", "зустріч", "встреча", "переговори", "переговоры",
    "вогонь", "огонь", "вода", "вода", "земля", "земля", "повітря", "воздух",
    "життя", "жизнь", "смерть", "смерть", "здоров'я", "здоровье", "залишки", "ліс", "лес", 

    # --- Generic Adjectives / Adverbs ---
    "новий", "новый", "старий", "старый", "великий", "большой", "малий", "малый", "маленький",
    "добрий", "добрый", "хороший", "гарний", "красивый", "поганий", "плохой",
    "можливий", "возможный", "ймовірний", "вероятный", "очевидний", "очевидный",
    "різний", "разный", "інший", "другой", "иной", "однаковий", "одинаковый", "схожий", "похожий",
    "загальний", "общий", "основний", "основной", "головний", "главный", "важливий", "важный",
    "певний", "определенный", "невизначений", "неопределенный", "відомий", "известный", "невідомий", "неизвестный",
    "військовий", "военный", "цивільний", "гражданский",
    "останній", "последний", "попередній", "предыдущий", "наступний", "следующий", 
    "правий", "правый", "лівий", "левый", "верхній", "верхний", "нижній", "нижний",
    "східний", "восточный", "західний", "западный", "північний", "северный", "південний", "южный",
    "швидко", "быстро", "повільно", "медленно", "добре", "хорошо", "погано", "плохо",
    "сильно", "сильно", "слабко", "слабо", "більше", "больше", "менше", "меньше", "краще", "лучше", "гірше", "хуже",
    "разом", "вместе", "окремо", "отдельно", "приблизно", "приблизительно",
    "майже", "почти", "дуже", "очень", "надто", "слишком", "достатньо", "достаточно", "особливо", "особенно", "звичайно", "обычно",
    "переважно", "преимущественно", "насправді", "на самом деле", "дійсно", "действительно",
    "зокрема", "в частности", "наприклад", "например",

    # --- Numbers / Quantifiers 
    "один", "одна", "одне",
    "два", "дві",
    "три",
    "чотири", "четыре",
    "п'ять", "пять", "шість", "шесть", "сім", "семь", "вісім", "восемь", "дев'ять", "девять", "десять",
    "нуль", "ноль", "декілька", "несколько", "багато", "много", "мало", "кілька", "несколько",
    "пара", "сотня", "тисяча", "тысяча", "мільйон", "миллион",
    "перший", "первый", "другий", "второй", "третій", "третий", "четвертий", "четвертый", "п'ятий", "пятый",
    "раз", "рази", "раза",

    "берег", "катер", "острів", "остров","імовірність", "вероятность","з", "из","слово", "Міноборони", "Минобороны","вимикати", "выключать",
    "підрозділ", "подразделение","скупчення", "скопление","підписувати", "подписывать","над", "на""повторний", "повторный","прямий", "прямой",
    "місія", "миссия","передавати", "передавать","береговий", "береговой","лінія", "линия","мінімум", "минимум","департамент","захист", "защита",
    "війна", "война","ЗСУ","Генштаб","журналіст", "журналист","уточнення", "уточнение","склад","міністерство", "министерство","оборона",
    "зв'язок", "связь","окупант", "оккупант","обіцяти", "обещать","компанія", "компания","загалом", "в общем", "в целом","інфраструктура",
    "пожежа", "пожар","внаслідок", "вследствие","поранити", "ранить","жінка", "женщина","голова", "глава","ОВА","президент","збирати", "собирать",
    "супутник", "спутник","одиниця", "единица","застосунок", "приложение","опублікувати", "опубликовать","супутниковий", "спутниковый","знімок", "снимок",
    "водосховище", "водохранилище","енергоблок","черга", "очередь","відповідь", "ответ","ваш","зупинитися", "остановиться","житловий", "жилой",
    "відповідний", "соответствующий","розділитися", "разделиться","фонд","відкрити", "открыть","збір", "сбор","повз", "мимо","більшість", "большинство"
}


stop_words.update(custom_stops)


In [56]:
def preprocess_text(text):
    text = ftfy.fix_text(text)

    # # author line patterns
    # text = re.sub(r"Russian Offensive Campaign Assessment,.*?\d{1,2}:\d{2}\s*(?:am|pm)\s*ET", "", text, flags=re.IGNORECASE)
    # # common map links
    text = re.sub(r"https://hromadske.ua/posts*?\.", "", text, flags=re.IGNORECASE)
    # bracketed numbers
    text = re.sub(r'\[\d+\]', '', text)

    text = re.sub(r'((([A-Za-z]{3,9}:(?://)?)(?:[-;:&=\+\$,\w]+@)?[A-Za-z0-9.-]+|(?:www.|[-;:&=\+\$,\w]+@)[A-Za-z0-9.-]+)((?:/[\+~%/.\w_-]*)?\??(?:[-\+=&;%@.\w_]*)#?(?:[.\!/\\\w]*)))', '', text)

    text = text.lower()

    # punctuation
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s-]', '', text)
    text = re.sub(r'\s-\s|\s-$|^-', ' ', text)

    # tokenization
    tokens = word_tokenize(text)

    # lemmatization
    processed_tokens = [
        lemmatizer.lemmatize(word) for word in tokens
        if word not in stop_words and len(word) > 2 and not word.startswith('-') and not word.endswith('-')
    ]

    return ' '.join(processed_tokens)

In [57]:
df['processed_text2'] = df['message'].apply(preprocess_text)

In [58]:
df

,date,message,time,processed_text2
0,2022-02-24,⚡️Путін [заявив](https://hromadske.ua/posts/pu...,23:01:19,путін заявив спеціальної військової операції б...
1,2022-02-25,[​​](https://telegra.ph/file/f7bd512a24f6e63cb...,23:56:20,києва заявляє трьох поранених потрапляння улам...
2,2022-02-26,⚡️У Києві за вчора [вбили](https://www.faceboo...,23:53:24,києві вбили диверсантів радник мвс віктор андр...
3,2022-02-27,⚡️У Харкові від обстрілів [постраждав](https:/...,23:57:28,харкові обстрілів постраждав будинок зруйнован...
4,2022-02-28,«Переговори підуть або в потрібному 🇺🇦 напрямк...,23:09:19,підуть або потрібному напрямку або російський ...
...,...,...,...,...
1092,2025-02-26,Останні два БпЛА на Полтавщині у Полтавському ...,23:07:22,останні бпла полтавщині полтавському районі за...
1093,2025-02-27,"02:35\n\nБпЛА меншає, більшість збивається в о...",23:55:08,бпла меншає збивається області бпла хмельниччи...
1094,2025-02-28,⚠️ Київ — залишає Подільський район вектором н...,23:24:12,київ залишає подільський вектором ірпінь остан...
1095,2025-03-01,02:10\n\nБпЛА значно поменшало.\n\n1х БпЛА на ...,23:39:09,бпла значно поменшало бпла вінниччині остання ...


In [59]:
df = df.drop(columns=['time', 'message'])

In [60]:
df

,date,processed_text2
0,2022-02-24,путін заявив спеціальної військової операції б...
1,2022-02-25,києва заявляє трьох поранених потрапляння улам...
2,2022-02-26,києві вбили диверсантів радник мвс віктор андр...
3,2022-02-27,харкові обстрілів постраждав будинок зруйнован...
4,2022-02-28,підуть або потрібному напрямку або російський ...
...,...,...
1092,2025-02-26,останні бпла полтавщині полтавському районі за...
1093,2025-02-27,бпла меншає збивається області бпла хмельниччи...
1094,2025-02-28,київ залишає подільський вектором ірпінь остан...
1095,2025-03-01,бпла значно поменшало бпла вінниччині остання ...


In [61]:
df.to_csv('telegram_DEV.csv', index=False)

In [2]:
df = pd.read_csv('telegram_DEV.csv')

In [5]:
df = df.rename(columns={"processed_text2": "processed_text"})

In [6]:
df

,date,processed_text
0,2022-02-24,путін заявив спеціальної військової операції б...
1,2022-02-25,києва заявляє трьох поранених потрапляння улам...
2,2022-02-26,києві вбили диверсантів радник мвс віктор андр...
3,2022-02-27,харкові обстрілів постраждав будинок зруйнован...
4,2022-02-28,підуть або потрібному напрямку або російський ...
...,...,...
1092,2025-02-26,останні бпла полтавщині полтавському районі за...
1093,2025-02-27,бпла меншає збивається області бпла хмельниччи...
1094,2025-02-28,київ залишає подільський вектором ірпінь остан...
1095,2025-03-01,бпла значно поменшало бпла вінниччині остання ...


In [7]:
# optimal values based on the analysis of the resulting vectorised data.
tfidf_vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),
    max_df=0.80,
    min_df=5,
    max_features=3000,
    sublinear_tf=True
)

tfidf_matrix = tfidf_vectorizer.fit_transform(df['processed_text'])

In [8]:
# initially we tended to use fewer max_features and not use TruncatedSVD, but it caused memory problems
# when working with a merged dataset. to solve this problem we decided to use TruncatedSVD,
# and to cover the potential loss of information with the increased the number of max_features

n_components = 30
svd = TruncatedSVD(n_components=n_components, random_state=1) # random_state=1
tfidf_svd_matrix_dense = svd.fit_transform(tfidf_matrix)

In [9]:
with open('tfidf_vectorizer_tg.pkl', 'wb') as f_tfidf:
    pickle.dump(tfidf_vectorizer, f_tfidf)

with open('svd_reducer_tg.pkl', 'wb') as f_svd:
    pickle.dump(svd, f_svd)

In [6]:
svd_feature_names = [f'svd2_comp_{i+1}' for i in range(n_components)]
df_tfidf_svd = pd.DataFrame(tfidf_svd_matrix_dense, columns=svd_feature_names, index=df.index)

In [7]:
df_tfidf_svd['date'] = df['date']
date_col = df_tfidf_svd.pop('date') # make the 'date' column appear first
df_tfidf_svd.insert(0, 'date', date_col)

In [8]:
df_tfidf_svd.to_csv('telegram_data30_3000_2.csv', index=False)